In [1]:
from datasets import load_dataset
from collections import defaultdict
ds = load_dataset('hotpotqa/hotpot_qa', 'fullwiki', trust_remote_code=True)

/home/junseokkim00/anaconda3/envs/rp2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ds_train_rm_distract
import random

k=100
indices = random.sample(range(len(ds['train'])), k)
test_ds = ds['train'].select(indices)

In [3]:
test_ds[0]

{'id': '5abbbdf355429931dba144d9',
 'question': 'Who voiced the character of the person who helped Ned Flanders spread the religious belief of creationism in the episode The Monkey Suit ?',
 'answer': 'Harry Shearer,',
 'type': 'bridge',
 'level': 'medium',
 'supporting_facts': {'title': ['The Monkey Suit', 'Reverend Lovejoy'],
  'sent_id': [3, 1]},
 'context': {'title': ['Young Earth creationism',
   'The Monkey Suit',
   'Progressive creationism',
   'Okilly Dokilly',
   'Reverend Lovejoy',
   "I'm Goin' to Praiseland",
   'Maintenance of Religious Harmony Act',
   'Alone Again, Natura-Diddily',
   'Viva Ned Flanders',
   'When Flanders Failed'],
  'sentences': [['Young Earth creationism (YEC) is a form of creationism, a religious belief which holds that the universe, Earth, and all life on Earth were created by direct acts of God less than 10,000 years ago.',
    " Its primary adherents are Christians who subscribe to a literal interpretation of the creation narrative in the Bible's

In [6]:
from utils import questionDecompose, evidenceExtractor, run_RAG
from LLMs import set_model
total_cnt=0
cnt=0
preds=[]
labels=[]
name="llama3-8b-8192"
model = set_model(name)
for data in test_ds:
    question, answer, sentences = data['question'], data['answer'], data['context']['sentences']
    print(f"{total_cnt+1}/{len(test_ds)} Question: {question}")
    subQuestions=questionDecompose(model, question)
    print(f"subQuestions: {subQuestions}")
    fine_grained_context=""
    relevance_num=set()
    context=""
    context_dict={}
    for idx, sent_ls in enumerate(sentences):
        context+=f"{idx}. "
        context+=" ".join(sent_ls)
        context_dict[idx] = " ".join(sent_ls)
        context+='\n\n'
    
    # extract evidence number
    for subQ in subQuestions:
        num = evidenceExtractor(model, context, context_dict, question)
        relevance_num.add(num)

    # generate find_grained_context
    relevance_num = list(relevance_num)
    for num in relevance_num:
        fine_grained_context+=context_dict[num]
        fine_grained_context+='\n\n'

    pred = run_RAG(model=model, context=fine_grained_context, question=question)
    preds.append(pred)
    labels.append(answer)
    print(f"pred: {pred}\tlabel: {answer}")
    if pred == answer:
        cnt+=1
    total_cnt+=1


ModuleNotFoundError: No module named 'utils'